### 데이터 불러오기

In [1]:
import pandas as pd

# Load the dataset
file_path = 'MBTI_500.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset and its basic information
data_info = data.info()
data_head = data.head()

data_info, data_head


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106067 entries, 0 to 106066
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   posts   106067 non-null  object
 1   type    106067 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


(None,
                                                posts  type
 0  know intj tool use interaction people excuse a...  INTJ
 1  rap music ehh opp yeah know valid well know fa...  INTJ
 2  preferably p hd low except wew lad video p min...  INTJ
 3  drink like wish could drink red wine give head...  INTJ
 4  space program ah bad deal meing freelance max ...  INTJ)

### Data Tokenization and train-test sets

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download stopwords from NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Define a function for preprocessing
def preprocess_text(texts):
    # Tokenize the text
    tokens = [word_tokenize(text.lower()) for text in texts]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_texts = [" ".join([word for word in token if word not in stop_words]) for token in tokens]
    
    return filtered_texts

# Apply preprocessing on a smaller subset to demonstrate
subset_size = 1000  # Using a smaller subset for demonstration
data_subset = data.sample(n=subset_size, random_state=42)

# Preprocess the text
preprocessed_texts = preprocess_text(data_subset['posts'])

# Vectorize the preprocessed text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Limiting to 1000 features for demonstration
X = tfidf_vectorizer.fit_transform(preprocessed_texts)
y = data_subset['type']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MoohyeonKim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MoohyeonKim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


((800, 1000), (200, 1000))

### Model Selection - Comparison of 5 models 

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Assuming X_train, X_test, y_train, y_test are already defined

# Initialize models
lr_model = LogisticRegression(max_iter=1000)
svm_model = LinearSVC(max_iter=1000)
nb_model = MultinomialNB()
rf_model = RandomForestClassifier(n_estimators=100)
gbm_model = GradientBoostingClassifier(n_estimators=100)

models = {
    "Logistic Regression": lr_model,
    "Support Vector Machine": svm_model,
    "Naive Bayes": nb_model,
    "Random Forest": rf_model,
    "Gradient Boosting": gbm_model
}

# Initialize an empty list to store results
results = []

# Train, predict, and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Predict on test set
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted', zero_division=0)
    
    # Append results to list
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": fscore
    })

# Convert the list of dictionaries to a DataFrame
results_df = pd.concat([pd.DataFrame([r]) for r in results], ignore_index=True)

# Display the results DataFrame
print(results_df)


c:\Users\MoohyeonKim\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


                    Model  Accuracy  Precision  Recall  F1 Score
0     Logistic Regression     0.645   0.605448   0.645  0.591326
1  Support Vector Machine     0.695   0.684150   0.695  0.678801
2             Naive Bayes     0.435   0.302888   0.435  0.332326
3           Random Forest     0.645   0.614778   0.645  0.605259
4       Gradient Boosting     0.690   0.718143   0.690  0.694243


결과적으로 SVM이나 GB를 쓰면 될 듯 함. 또는 이 둘의 앙상블 기법 활용?

### Next Steps:  
- Addressing Class Imbalance: If your dataset is imbalanced (some MBTI types have significantly more examples than others), consider techniques like SMOTE for oversampling the minority classes or adjusting class weights in the model.  
- Parameter Tuning: Experimenting with different hyperparameters for both the Logistic Regression and SVM models can potentially improve performance further.  
- Feature Engineering: Exploring different settings for TF-IDF vectorization (such as adjusting the number of features, using bigrams or trigrams) or even using word embeddings could yield better results.  
- Advanced Models: Consider trying more complex models or neural networks, such as those based on the transformer architecture, if you have the computational resources. These models might capture the nuances of language better but require careful tuning and more computational power.  

### Addressing Class Imbalance: Using a language model like GPT-3 or GPT-4 to generate synthetic text data that mimics the writing styles and content preferences of specific MBTI types

1. Analyze the Current Dataset Distribution  
First, identify which MBTI types are underrepresented in your dataset. This involves analyzing the distribution of MBTI types within your dataset to determine which types need more samples.

2. Prepare Seed Prompts for Text Generation  
For each underrepresented MBTI type, prepare seed prompts that capture the essence of how individuals with that MBTI type might express themselves. These prompts could be based on characteristics known to be associated with the MBTI type or derived from the existing samples in your dataset.

3. Use GPT for Text Generation  
With your seed prompts ready, use the OpenAI API to generate additional text samples. You can customize the prompts to encourage the generation of text that reflects specific aspects of the targeted MBTI personality.  
For example:  


In [ ]:
# import openai

# openai.api_key = 'your-api-key'

# response = openai.Completion.create(
#   engine="text-davinci-003", # or whichever engine you prefer
#   prompt="Write a reflective journal entry from the perspective of an INFP talking about their day.",
#   temperature=0.7,
#   max_tokens=150,
#   top_p=1.0,
#   frequency_penalty=0.0,
#   presence_penalty=0.0,
#   n=5 # Generate multiple samples per prompt
# )


4. Post-process and Validate Generated Text  
After generating the text, it's important to post-process and validate it to ensure it aligns well with the targeted MBTI type's characteristics. This step might involve manual review or could be assisted by classifiers trained to identify inconsistencies or off-target generations.

5. Integrate Synthetic Data  
Finally, integrate the synthetic data into your dataset, ensuring a more balanced distribution across MBTI types. Retrain your model on this augmented dataset and evaluate its performance to see if the class imbalance issue has been mitigated effectively.

Considerations  
Quality Control: Ensure the generated text is of high quality and accurately reflects the nuances of the targeted MBTI type. Poor-quality or irrelevant synthetic data could harm your model's performance.
Ethical Use: Be transparent about using synthetic data in your model training process, especially if the model's output will be used in real-world applications.

#### MBTI Distributions

In [4]:
# Analyze the distribution of MBTI types in the dataset
mbti_distribution = data['type'].value_counts(normalize=True) * 100

mbti_distribution


type
INTP    23.533238
INTJ    21.144182
INFJ    14.107121
INFP    11.439939
ENTP    11.054334
ENFP     5.814249
ISTP     3.228148
ENTJ     2.785975
ESTP     1.872401
ENFJ     1.446256
ISTJ     1.171901
ISFP     0.824950
ISFJ     0.612820
ESTJ     0.454430
ESFP     0.339408
ESFJ     0.170647
Name: proportion, dtype: float64

The least represented types: ESFJ, ESFP, ESTJ, and ISFJ (constituting less than 1% of the total entries.)  
The most represented types: INTP, INTJ, INFJ, and INFP

#### let's generate more samples for mbti types with less than 2%: ESTP, ENFJ, ISTJ, ISFP, ISFJ, ESTJ, ESFP

**Example Prompts for MBTI Type Text Generation**  
For each MBTI type, you would create a prompt that guides the GPT model to generate text reflecting that type's communication style, interests, and typical expression forms. Here's how you might structure these prompts:  

ESTP: "Write a short story about an ESTP experiencing an exciting adventure in the city, showcasing their spontaneous and action-oriented nature."  
ENFJ: "Compose a motivational speech from the perspective of an ENFJ, focusing on inspiring a team to achieve a common goal, demonstrating their empathetic and leadership qualities."  
ISTJ: "Draft an email from an ISTJ planning a detailed and structured family reunion, highlighting their organization skills and dedication to tradition."  
ISFP: "Describe a day in the life of an ISFP artist, capturing their creative process, love for beauty, and preference for expressing themselves through art."  
ISFJ: "Write a diary entry from an ISFJ volunteering at a local community center, reflecting on the day's events and their feelings of satisfaction from helping others."  
ESTJ: "Create a detailed plan from an ESTJ organizing a corporate event, emphasizing their leadership, efficiency, and practical problem-solving approach."  
ESFP: "Tell a story about an ESFP throwing an impromptu party for their friends, focusing on their spontaneity, love for social gatherings, and ability to live in the moment."  
ESFJ: "Write a letter from an ESFJ to a friend going through a tough time, offering support and advice, showcasing their caring, sociable, and supportive nature."  

**Generating Text with the OpenAI API**  
For each prompt, you would use the OpenAI API similar to the following command, adjusting the prompt parameter accordingly:

In [ ]:
# response = openai.Completion.create(
#   engine="text-davinci-003",
#   prompt="PROMPT_GOES_HERE",
#   temperature=0.7,
#   max_tokens=200,
#   top_p=1.0,
#   frequency_penalty=0.0,
#   presence_penalty=0.0,
#   n=5 # Number of samples to generate
# )


**Integrating Generated Samples**  
After generating the samples, carefully review them to ensure they accurately reflect the intended MBTI type's characteristics. You may need to manually verify the quality and relevance of the generated text to ensure it's suitable for training your model.  

Once satisfied, append these samples to your dataset, ensuring to label them correctly with their MBTI type. This augmented dataset should then provide a more balanced representation of MBTI types, potentially improving your model's classification performance across the board.

#### Advice on integrating generated samples:  
##### Integrating Generated Samples
**Review and Clean**: After generating the text samples, it's crucial to review them for relevance and quality. Ensure that the generated text aligns with the characteristics and communication style of the targeted MBTI type. Remove any samples that are off-topic, repetitive, or do not meet the quality standards.  

**Label Appropriately**: Assign the correct MBTI type label to each generated sample. This step is crucial for maintaining the integrity of your dataset and ensuring that the model learns the correct associations between text patterns and MBTI types.  

**Balance Your Dataset**: Aim for a balanced representation of MBTI types in your dataset. While perfect balance may not be achievable or necessary, reducing the skewness can help improve model performance across all types.  

**Split Your Dataset**: After integrating the generated samples, split your dataset into training, validation, and test sets. This split is crucial for evaluating your model's performance on unseen data.  

##### Refining Prompts  
**Target MBTI Characteristics**: Make sure your prompts are specifically designed to elicit responses that reflect the unique traits of each MBTI type. Researching each type's common behaviors, interests, and communication styles can help craft more effective prompts.  

**Vary Prompt Types**: Use a variety of prompt types to generate a diverse set of responses. For instance, besides reflective journal entries or stories, consider prompts that ask for opinions on topics, describe reactions to hypothetical scenarios, or involve planning an event. This variety can help capture a broader spectrum of each type's characteristics.  

**Adjust Parameters**: Experiment with different settings for the temperature, max_tokens, and top_p parameters to control the creativity, length, and diversity of the generated responses. A lower temperature (e.g., 0.5-0.7) tends to produce more coherent and predictable text, while a higher temperature (e.g., 0.8-1.0) generates more diverse and creative content.  

**Iterative Refinement**: It's a process of trial and error. Generate a small batch of samples, evaluate their quality and relevance, and adjust your prompts based on your findings. This iterative process can help you fine-tune the prompts to produce more accurate and high-quality text samples.  

##### Post-Generation Processing  
**Augmentation**: Consider using text augmentation techniques on both original and generated samples to further increase the diversity of your dataset. Techniques such as synonym replacement or sentence restructuring can provide additional variance.  

**Quality Control Mechanism**: Establish a quality control mechanism, possibly involving manual review or a secondary classifier, to ensure that the generated text meets your criteria for inclusion in the training dataset.  

**Ethical Considerations**: When generating and using synthetic text, consider the ethical implications, including transparency about the use of generated text and ensuring that the generated content does not perpetuate biases or stereotypes.  

By following these guidelines, you can effectively integrate GPT-generated samples into your dataset to address class imbalance and refine your prompts to ensure the generation of high-quality, relevant text for each MBTI type. This approach can enhance your model's ability to classify MBTI types accurately by providing a richer and more balanced training dataset.

#### Full Code Example for Text Generation

In [ ]:
import openai

# Replace "your_api_key_here" with your actual OpenAI API key
openai.api_key = 'your_api_key_here'

def generate_text(prompt, n=5, temperature=0.7, max_tokens=200):
    """
    Generate text using the OpenAI GPT model.

    :param prompt: The prompt to generate text for.
    :param n: Number of text samples to generate.
    :param temperature: Controls the creativity of the output. Higher values mean more creative responses.
    :param max_tokens: The maximum number of tokens to generate in the output.
    :return: A list of generated text samples.
    """
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",  # Choose the model version
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            n=n
        )
        return [completion['text'].strip() for completion in response['choices']]
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example prompt for an ESTP type
prompt_estp = "Imagine you're an ESTP talking about your latest adventure with friends. Describe the thrill of the moment and how you navigated the challenges you faced."

# Generate text for the ESTP prompt
generated_texts_estp = generate_text(prompt=prompt_estp, n=5)

for i, text in enumerate(generated_texts_estp, start=1):
    print(f"Sample {i}:\n{text}\n")

#### 코드 설명:
**Customizing the Function**  
prompt: Replace this with any of the MBTI-specific prompts you've prepared.  
n: Adjust this to control how many text samples you want to generate per prompt. Generating multiple samples can give you a broader range of responses to evaluate and select from.  
temperature: This parameter controls the creativity of the responses. A higher temperature results in more varied and creative outputs, while a lower temperature produces more deterministic and possibly coherent outputs.  
max_tokens: This defines the maximum length of the generated text. Adjust this based on how long you want your samples to be.  

**Running the Code**  
To generate text for different MBTI types, replace the prompt_estp variable with the appropriate prompt for the MBTI type you're focusing on. This code snippet will print out the generated text samples, which you can then review and select for inclusion in your dataset.

### Hyperparameter Tuning

optimizing the performance of machine learning models

#### Hyperparameter Tuning for Gradient Boosting

use GridSearchCV from sklearn.model_selection to systematically work through multiple combinations of parameter tunes, cross-validating as it goes to determine which tune gives the best performance.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# Define the parameter grid to search
param_grid_gbm = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the model
gbm = GradientBoostingClassifier()

# Initialize the GridSearchCV object
grid_search_gbm = GridSearchCV(estimator=gbm, param_grid=param_grid_gbm, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the grid search to the data
grid_search_gbm.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters for Gradient Boosting:", grid_search_gbm.best_params_)
print("Best Score for Gradient Boosting:", grid_search_gbm.best_score_)


#### Hyperparameter Tuning for SVM

Similarly, for SVM, you can use GridSearchCV to explore different configurations. Since LinearSVC does not directly expose the kernel parameter, if you're specifically interested in experimenting with kernel SVMs, consider using SVC from sklearn.svm.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameter grid to search
param_grid_svm = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Type of SVM kernel
    'degree': [2, 3, 4],  # Degree of the polynomial kernel function (if 'poly' is chosen)
    'gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
}

# Initialize the model
svm = SVC()

# Initialize the GridSearchCV object
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the grid search to the data
grid_search_svm.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters for SVM:", grid_search_svm.best_params_)
print("Best Score for SVM:", grid_search_svm.best_score_)


#### Tips for Hyperparameter Tuning

Computational Cost: Hyperparameter tuning, especially grid search with many parameters and large datasets, can be computationally expensive. Consider starting with a smaller subset of your data or a more limited set of parameter values to get a sense of performance trends.  

Cross-Validation: Using cross-validation (cv parameter in GridSearchCV) helps ensure that the chosen hyperparameters generalize well to unseen data.  

Scoring Metric: Ensure the scoring parameter in GridSearchCV aligns with your project goals (e.g., 'accuracy', 'f1_weighted'). Different metrics will lead to different tuning results.  

Parallelization: Setting n_jobs=-1 utilizes all available CPUs to perform the searches in parallel, speeding up the grid search process.  

### Feature Engineering

**Word Embeddings**: Replace TF-IDF with word embeddings like Word2Vec, GloVe, or fastText. These models can capture semantic meanings of words and are particularly powerful for NLP tasks. You can use pre-trained embeddings or train your own on your dataset.  

**n-grams**: Expanding your feature space to include bi-grams or tri-grams (sequences of two or three words) can help capture more context than single words alone, although this will increase the dimensionality of your data.  

**Part-of-Speech Tagging**: Adding features based on the part of speech (e.g., noun, verb, adjective) of words in your texts might help the model learn more about the syntactic structure of sentences.  

**Sentiment Analysis**: Incorporate features that capture the sentiment of the text. This can be particularly useful if certain MBTI types are more prone to expressing positive or negative sentiments.  

#### Word2Vec 사용하기 - 귀찮을거같음 ㅋㅋㅋ  

Training Your Own Word2Vec Model  
If you prefer to train a Word2Vec model on your own dataset to capture domain-specific semantics, you can do so using Gensim.  

Download Pre-trained Word2Vec: Google's pre-trained Word2Vec model is trained on part of the Google News dataset. It contains 300-dimensional vectors for 3 million words and phrases. You can download it from the official Google Code Archive.

1. Prepare Your Dataset: Tokenize your text data into a list of words for each document.  

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Assuming `data['posts']` is your text column
sentences = [word_tokenize(document.lower()) for document in data['posts']]

2. Train Word2Vec Model: Use Gensim to train a model on your processed dataset.

In [ ]:
from gensim.models import Word2Vec

# Train a Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

3. Create an Embedding Matrix: After training, create an embedding matrix as you would with pre-trained embeddings.

In [ ]:
embedding_dim = 100  # Match the `vector_size` parameter from the model
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue  # Word not in model


4. Use Embedding Matrix in Neural Network: The embedding matrix can then be used in the embedding layer of a neural network model

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Flatten())  # Or consider more complex layers like LSTM, GRU, or Conv1D
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=len(set(y_train)), activation='softmax'))  # Assuming y_train contains your labels

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Machine learning model -> Advanced Model : BERT 사용하기

코랩 T4 GPU 사용한다는 가정

Example Steps (Skeleton) generated by GPT

In [7]:
# Step 1: Prepare the Environment
!pip install transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from tqdm import tqdm

In [ ]:
# Step 2: Load and Preprocess the Data
# Load the dataset and preprocess it. Preprocessing includes encoding the MBTI types into numerical labels.

# Load the dataset
data_path = "/mnt/data/MBTI_500.csv"
data = pd.read_csv(data_path)

# Encode MBTI types
label_encoder = LabelEncoder()
data['type_encoded'] = label_encoder.fit_transform(data['type'])

# Split the data
X_train, X_val, y_train, y_val = train_test_split(data['posts'], data['type_encoded'], test_size=0.1, random_state=42)

In [ ]:
# Step 3: Tokenize the Text Data
# Tokenize the text using BERT's tokenizer.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_data(tokenizer, texts, max_length=128):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

max_length = 128
train_inputs, train_masks = encode_data(tokenizer, X_train, max_length)
val_inputs, val_masks = encode_data(tokenizer, X_val, max_length)

In [ ]:
# Step 4: Create Data Loaders

batch_size = 32

train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_inputs, val_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# Step 5: Load BERT Model for Sequence Classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_encoder.classes_),
    output_attentions=False,
    output_hidden_states=False,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Step 6: Set Up the Training Loop

#Define Optimizer & Learning Rate Scheduler
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,  # Learning rate
                  eps = 1e-8  # Epsilon
                 )

total_steps = len(train_dataloader) * epochs  # Number of training epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# Training & Validation Loop
import numpy as np
from sklearn.metrics import f1_score

# Specify the number of epochs
epochs = 4

# Function to calculate the accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad()        

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs.logits

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))


생각보다 어렵다.. 

### 이제 Train 된 모델 (ML이든 BERT든)을 챗봇으로

1. **Integrate Model with a Chat Interface**  
You'll need a way to interact with users in real-time. This can be done through various platforms, such as a web application, a messaging app (like Telegram, Slack, or Discord), or even a simple command-line interface. Choose the platform that best fits your target audience and the resources you have available.

- **Web Application**: Use frameworks like Flask or Django in Python to create a web interface where users can chat with the bot.
- **Messaging Apps**: Platforms like Telegram and Slack offer APIs to build bots that can respond to user messages.
- **Command-Line Interface (CLI)**: For a simple prototype, a CLI chatbot can be developed where users input their responses directly in the terminal.
2. **Preprocess User Input**
Just as you preprocessed your training data, user inputs need to be preprocessed before making predictions. This includes tokenization, removing stopwords (if your model requires this), and converting inputs into the format expected by your model (e.g., sequences of tokens for BERT).

3. **Make Predictions and Interpret Results**
Once the user input is preprocessed and formatted correctly, use your trained model to predict the MBTI type. For BERT and other transformer-based models, this means running the input through the model and interpreting the output logits. For SVM, GB, and other traditional models, the input needs to be vectorized (e.g., with TF-IDF) before making predictions.

4. **Respond to the User**
Based on the predicted MBTI type, craft responses that the chatbot can return to the user. These responses can include insights about the predicted MBTI type, further questions to refine the prediction, or any other relevant information.

5. **Loop and Refine**
The chatbot should allow for multiple interactions, refining its predictions based on cumulative responses or simply engaging the user in a conversation about their MBTI type.

#### Example Workflow for a BERT-based Chatbot

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the trained model (Assuming it's already trained and saved)
model = BertForSequenceClassification.from_pretrained('path_to_saved_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def classify_mbti(text):
    # Preprocess the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Interpret the prediction
    prediction = torch.argmax(outputs.logits, dim=1).item()
    
    # Convert prediction to MBTI type (assuming you have a mapping)
    mbti_type = label_encoder.inverse_transform([prediction])[0]
    
    return mbti_type

# Example chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    
    mbti_type = classify_mbti(user_input)
    print(f"Chatbot: Based on your response, your MBTI type might be {mbti_type}.")


Remember, for SVM or GB models, you'll need to vectorize the user input using the same transformation applied to the training data (e.g., TF-IDF) before making predictions.

**Final Touches**  

**User Experience**: Consider the user experience. Ensure responses are friendly, informative, and engaging.  
**Testing and Iteration**: Test your chatbot extensively to refine its accuracy and user interaction. Gather feedback to make improvements.  
**Deployment**: Choose a deployment platform. Cloud services like AWS, GCP, and Azure offer scalable options for deploying models and applications.